In [20]:
from cartPoleAgent import Agent, GRUAgent
from gymnasium.envs.classic_control.cartpole import CartPoleEnv
import cartPoleEnv
import gymnasium as gym
import torch.nn as nn
import torch
import numpy as np

In [15]:
envs = gym.vector.SyncVectorEnv(
        [cartPoleEnv.make_env("CartPole-v1", i, True, "testRun") for i in range(2)],)

/opt/miniconda3/envs/metaRL/lib/python3.10/site-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /Users/constantinpinkl/University/Research/dsLab/robodogs/cartPole/basicExperiments/videos/testRun folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [76]:
gruAgent = GRUAgent(envs)

In [102]:
gru = nn.GRU(
    input_size=4,
    hidden_size=128,
    batch_first=True
)

In [103]:
next_obs, _ = envs.reset(seed=42)
next_obs.shape

(2, 4)

In [107]:
gru = nn.GRU(
    input_size=4,
    hidden_size=128,
    batch_first=True
)
full_h, l_h = gru(torch.tensor(next_obs).reshape(2, 1, 4), torch.zeros(1, 2, 128))
gru(torch.tensor(next_obs).reshape(2, 1, 4), l_h)[1].shape

#full_h, l_h = gru(torch.tensor(envs.single_observation_space.sample()).reshape(1, -1), full_h)
# gru(torch.tensor(envs.single_observation_space.sample()).reshape(1, -1), full_h)[0].shape

torch.Size([1, 2, 128])

In [21]:
mlp =  nn.Sequential(
            (nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
            nn.Tanh(),
            (nn.Linear(64, 64)),
            nn.Tanh(),
            (nn.Linear(64, 1)),
        )

In [49]:
mlp(torch.tensor(envs.single_observation_space.sample()))

tensor([0.1086], grad_fn=<ViewBackward0>)

In [71]:
# Online step-by-step (seq_len stays 1 each call)
h = torch.zeros(1, 1, 128)
obs = []
for t in range(10):
    obs.append(torch.tensor(envs.single_observation_space.sample()))
    x_t = obs[-1].reshape(1,1, -1)        # (B,1,obs_dim)
    print(h.shape)
    out_t, h = gru(x_t, h)           # out_t: (B,1,H)
    # store out_t if you want history

# Batched sequence (seq_len = T in one call)
X = torch.stack([obs[t] for t in range(10)], dim=0)  # (B,T,obs_dim)
print(X.unsqueeze(0).shape)
out, h = gru(X.unsqueeze(0))                                     # out: (B,T,H)
print(out.shape)


torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 1, 128])
torch.Size([1, 10, 4])
torch.Size([1, 10, 128])


In [72]:
obs = torch.zeros((100, 6) + envs.single_observation_space.shape)

In [73]:
obs.shape

torch.Size([100, 6, 4])